In [37]:
import pandas as pd
import re
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [38]:
spark = SparkSession.builder.appName('Read CSV File into DataFrame')\
    .config('spark.executor.memory', '10g') \
    .config('spark.driver.memory', '15g')\
    .config('spark.dirver.maxResultSize', '4096') \
    .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [39]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([StructField("polarity", IntegerType()),
                   StructField("title", StringType()),
                   StructField("text", StringType())
                  ])

In [40]:
path = "./amazon_review_polarity_csv/"

df_train = spark.read.format("csv").schema(schema).load(path + 'train.csv')
df_test = spark.read.format("csv").schema(schema).load(path + 'test.csv')

In [41]:
df = df_train.union(df_test)
df_test.show(3)

+--------+--------------------+--------------------+
|polarity|               title|                text|
+--------+--------------------+--------------------+
|       2|            Great CD|"My lovely Pat ha...|
|       2|One of the best g...|Despite the fact ...|
|       1|Batteries died wi...|I bought this cha...|
+--------+--------------------+--------------------+
only showing top 3 rows



In [42]:
df.limit(2).toPandas()

,polarity,title,text
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...


### Transform

#### Tokenization

In [43]:
tokenizer = Tokenizer(inputCol  = "text",
                      outputCol = "tokens")

df1 = tokenizer.transform(df)
df1.limit(2).toPandas()

,polarity,title,text,tokens
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...,"[this, sound, track, was, beautiful!, it, pain..."
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...,"[i'm, reading, a, lot, of, reviews, saying, th..."


#### Remove Punctuation

In [44]:
def normalize(tokens : list) -> list:
    """
    Removes non-english characters and returns lower case versions of words.
    """
    subbed   = [re.sub("[^a-zA-Z]+", "", s).lower() for s in tokens]

    filtered = filter(None, subbed)

    return list(filtered)

normalizeUDF = F.udf(normalize, T.ArrayType(StringType()))

In [45]:
def removeRegex(tokens: list) -> list:
    """
    Removes hashtags, call outs and web addresses from tokens.
    """
    expr    = '(@[A-Za-z0-a9_]+)|(#[A-Za-z0-9_]+)|'+\
              '(https?://[^\s<>"]+|www\.[^\s<>"]+)'

    regex   = re.compile(expr)

    cleaned = [t for t in tokens if not(regex.search(t)) if len(t) > 0]

    return list(filter(None, cleaned))

removeWEBUDF = F.udf(removeRegex, T.ArrayType(StringType()))

In [46]:
df2 = df1.withColumn("tokens_re", removeWEBUDF(df1["tokens"]))
# remove non english characters
df3 = df2.withColumn("tokens_clean", normalizeUDF(df2["tokens_re"]))
# rename columns
df4 = df3.drop("token","tokens_re")
df4 = df4.withColumnRenamed("tokens_clean", "tokens_new")
# remove tweets where the tokens array is empty, i.e. where it was just
# a hashtag, callout, numbers, web adress etc.

In [50]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="tokens_new", outputCol="tokens_filtered")
df4 = remover.transform(df4)

In [51]:
df4.printSchema()

root
 |-- polarity: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tokens_new: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tokens_filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)



### Model

In [35]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [36]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

# get the name of the metric used
evaluator.getMetricName()

'areaUnderROC'

In [54]:
train, test = df4.randomSplit([0.70, 0.30], 1234)

In [55]:
train.cache()

DataFrame[polarity: int, title: string, text: string, tokens: array<string>, tokens_new: array<string>, tokens_filtered: array<string>]

In [58]:
# train.groupby("polarity")\
#      .count()\
#      .show()

In [57]:
test.cache()

DataFrame[polarity: int, title: string, text: string, tokens: array<string>, tokens_new: array<string>, tokens_filtered: array<string>]

In [59]:
# test.groupby("polarity")\
#     .count()\
#     .show()

In [62]:
# create term frequencies for each of the tokens
tf1 = HashingTF(inputCol="tokens_filtered", outputCol="rawFeatures", numFeatures=1e5)

# create tf-idf for each of the tokens
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=2.0)

# create basic logistic regression model
lr = LogisticRegression(maxIter=20)

# create entire pipeline
basic_pipeline = Pipeline(stages=[tf1, idf, lr])


In [63]:
model1 = basic_pipeline.fit(train)

Py4JJavaError: An error occurred while calling o1315.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 32.0 failed 1 times, most recent failure: Lost task 7.0 in stage 32.0 (TID 168) (yimingL.lan executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions

In [ ]:
predictions1 = model1.transform(test)
score1 = evaluator.evaluate(predictions1)

print("AUC SCORE: {}".format(score1))

In [ ]:
predictedAndLabels = predictions1.select(["prediction","label"])\
                                 .rdd.map(lambda r : (float(r[0]), float(r[1])))

from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(predictedAndLabels)

print("Test Set Accuracy: {}".format(metrics.accuracy))

In [18]:
# # Basic sentence tokenizer
# tokenizer = Tokenizer(inputCol="comment_text", outputCol="tokens")
# wordsData = tokenizer.transform(df_train)

In [19]:
# # Count the words in a document
# hashingTF = HashingTF(inputCol="tokens", outputCol="rawFeatures")
# tf = hashingTF.transform(wordsData)

In [22]:
# # Build the idf model and transform the original token frequencies into their tf-idf counterparts
# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idfModel = idf.fit(tf)
# tfidf = idfModel.transform(tf)

Py4JJavaError: An error occurred while calling o399.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 12.0 failed 1 times, most recent failure: Lost task 7.0 in stage 12.0 (TID 57) (yimingL.lan executor driver): org.apache.spark.SparkException: Failed to execute user defined function (Tokenizer$$Lambda$2858/1018312554: (string) => array<string>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.has

In [13]:
REG = 0.1
lr = LogisticRegression(featuresCol="features", labelCol='toxic', regParam=REG)
lrModel = lr.fit(tfidf)
res_train = lrModel.transform(tfidf)

AttributeError: 'IDF' object has no attribute '_jdf'